<a href="https://colab.research.google.com/github/hawc2/Text-Analysis-with-Python/blob/master/notebooks/ocr/BERT_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resources

Tutorial on BERT OCR: 

https://www.statestitle.com/resource/using-nlp-bert-to-improve-ocr-accuracy/

Related: https://medium.com/towards-artificial-intelligence/using-tesseract-ocr-for-text-recognition-with-google-colab-1c4513b9d3e0

# Libraries


In [1]:
!pip install pytesseract transformers torch
!pip install pyenchant

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import os
import sys

in_colab = "google.colab" in sys.modules or os.environ.get("COLAB_RELEASE_TAG")
if in_colab:
    !sudo apt-get -y install tesseract-ocr libtesseract-dev
    !sudo apt-get -y install libenchant-2-2
else:
    print("Skipping apt-get installs; use Homebrew on macOS.")

Skipping apt-get installs; use Homebrew on macOS.


In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /Users/alexwermer-
[nltk_data]     colan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alexwermer-colan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/alexwermer-colan/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/alexwermer-
[nltk_data]     colan/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
from PIL import Image
import pytesseract
from pytesseract import image_to_string
import torch
from transformers import BertTokenizer, BertForMaskedLM
import re
import nltk
from difflib import SequenceMatcher

/Users/alexwermer-colan/Code/Hawc2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
# this maybe fixed pyenchant install: https://github.com/pyenchant/pyenchant/issues/214
try:
    import enchant
    from enchant.checker import SpellChecker
except ImportError as exc:
    SpellChecker = None
    print(f"pyenchant not available: {exc}")

pyenchant not available: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details



# File Import

In [6]:
try:
    from google.colab import files
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
except Exception:
    print("Colab upload skipped (not in Colab).")

Colab upload skipped (not in Colab).


In [7]:
filename = '19369615323_Page_134.png'
try:
    _ = pytesseract.get_tesseract_version()
    text = image_to_string(Image.open(filename))
    text_original = str(text)
    print(text_original)
except Exception as exc:
    print(f"Skipping OCR; Tesseract or file missing: {exc}")
    text = ""
    text_original = ""

Skipping OCR; Tesseract or file missing: tesseract is not installed or it's not in your PATH. See README file for more information.


# Clean Text

In [8]:
# cleanup text
rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ', "n't": " not" , "'ll": " will", '*':' * ', 
        '(': ' ( ', ')': ' ) ', "s'": "s '"}

In [9]:
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)

def get_personslist(text):
    personslist = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))

try:
    personslist = get_personslist(text)
except LookupError:
    print("NLTK data missing for person extraction; skipping names list.")
    personslist = []
ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', "'"]


NLTK data missing for person extraction; skipping names list.


In [10]:
# using enchant.checker.SpellChecker, identify incorrect words
# maybe replace SpellChecker
if SpellChecker is None:
    print("Skipping spellcheck; pyenchant not available.")
    incorrectwords = []
    suggestedwords = []
else:
    d = SpellChecker("en_US")
    words = text.split()

    incorrectwords = [w for w in words if not d.check(w) and w not in ignorewords]
    # using enchant.checker.SpellChecker, get suggested replacements
    suggestedwords = [d.suggest(w) for w in incorrectwords]

    # replace incorrect words with [MASK]
    for w in incorrectwords:
        text = text.replace(w, '[MASK]')
        text_original = text_original.replace(w, '[MASK]')


Skipping spellcheck; pyenchant not available.


In [11]:
print(text)

# Load BERT Model

In [12]:
# Load, train and predict using pre-trained model
if not text or not text.strip():
    print("Skipping BERT predictions; no OCR text available.")
    predictions = None
    MASKIDS = []
else:
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_text = tokenizer.tokenize(text)
    if not tokenized_text:
        print("Skipping BERT predictions; tokenizer produced no tokens.")
        predictions = None
        MASKIDS = []
    else:
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']
        # Create the segments tensors
        segs = [i for i, e in enumerate(tokenized_text) if e == '.']
        segments_ids = []
        prev = -1
        for k, s in enumerate(segs):
            segments_ids = segments_ids + [k] * (s - prev)
            prev = s
        segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
        if len(segments_ids) != len(tokenized_text):
            segments_ids = [0] * len(tokenized_text)
        segments_tensors = torch.tensor([segments_ids])
        # prepare Torch inputs
        tokens_tensor = torch.tensor([indexed_tokens])
        # Load pre-trained model
        model = BertForMaskedLM.from_pretrained('bert-base-uncased')
        model.eval()
        # Predict all tokens
        with torch.no_grad():
            try:
                outputs = model(tokens_tensor, token_type_ids=segments_tensors)
                predictions = outputs.logits
            except Exception as exc:
                print(f"Skipping BERT predictions; {exc}")
                predictions = None


Skipping BERT predictions; no OCR text available.


With Peter's dataset: https://stackoverflow.com/questions/56010551/pytorch-embedding-index-out-of-range

# Refine BERT predictions

In [13]:
#Predict words for mask using BERT; 
#refine prediction by matching with proposals from SpellChecker
def predict_word(text_original, predictions, maskids):
    pred_words=[]
    for i in range(len(MASKIDS)):
        preds = torch.topk(predictions[0, MASKIDS[i]], k=50) 
        indices = preds.indices.tolist()
        list1 = tokenizer.convert_ids_to_tokens(indices)
        list2 = suggestedwords[i]
        simmax=0
        predicted_token=''
        for word1 in list1:
            for word2 in list2:
                s = SequenceMatcher(None, word1, word2).ratio()
                if s is not None and s > simmax:
                    simmax = s
                    predicted_token = word1
        text_original = text_original.replace('[MASK]', predicted_token, 1)
    return text_original
text_original = predict_word(text_original, predictions, MASKIDS)
print (text_original)